In [1]:
print('start tvl trends')
import pandas as pd
import sys
import numpy as np
import json

from datetime import datetime, timedelta
sys.path.append("../helper_functions")
import defillama_utils as dfl
import duneapi_utils as du
import opstack_metadata_utils as ops
sys.path.pop()

current_utc_date = datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)
# print(current_utc_date)

start tvl trends


In [2]:
categorized_chains = dfl.get_chains_config()

In [3]:
# Convert float values in 'parent' column to dictionaries
categorized_chains['parent'] = categorized_chains['parent'].apply(lambda x: {} if pd.isna(x) else x)

# Check if the chain belongs to EVM category
categorized_chains['is_EVM'] = categorized_chains['categories'].apply(lambda x: isinstance(x, list) and 'EVM' in x)

# Check if the chain belongs to Rollup category or L2 is in parent_types
def is_rollup(row):
    categories = row['categories']
    parent = row.get('parent')
    parent_types = parent.get('types') if parent else []
    
    if isinstance(categories, list) and 'Rollup' in categories:
        return True
    if 'L2' in parent_types:
        return True
    
    return False

def extract_layer(row):
    parent = row.get('parent')
    parent_types = parent.get('types') if parent else []
    
    if 'L2' in parent_types:
        return 'L2'
    elif 'L3' in parent_types:
        return 'L3'
    
    return 'L1'

categorized_chains['is_Rollup'] = categorized_chains.apply(is_rollup, axis=1)
categorized_chains['layer'] = categorized_chains.apply(extract_layer, axis=1)

#Replace opBNB
categorized_chains['chain'] = categorized_chains['chain'].replace('Op_Bnb', 'opBNB')

considered_chains = categorized_chains[categorized_chains['is_EVM']]
considered_chains = considered_chains[['chain','layer','is_EVM','is_Rollup']]

# considered_chains

In [4]:
considered_chains

,chain,layer,is_EVM,is_Rollup
0,Ethereum,L1,True,False
1,Arbitrum,L2,True,True
2,Palm,L1,True,False
3,Optimism,L2,True,True
9,BSC,L1,True,False
...,...,...,...,...
320,Map,L1,True,False
321,Pulse,L1,True,False
322,WEMIX,L1,True,False
324,TomoChain,L1,True,False


In [5]:
min_tvl_to_count_apps = 0

In [6]:
# OPStack Metadata will auto pull, but we can also add curated protocols here (we handle for dupes)
curated_chains = [
    #L2s
     ['Optimism', 'L2']
    ,['Base', 'L2']
    ,['Arbitrum', 'L2']
    #non-dune L2s - Check L2Beat for comprehensiveness
    ,['ZkSync Era', 'L2']
    ,['Polygon zkEVM', 'L2']
    ,['Starknet', 'L2']
    ,['Linea', 'L2']
    ,['Mantle', 'L2']
    ,['Scroll', 'L2']
    ,['Boba', 'L2']
    ,['Metis', 'L2']
    ,['opBNB', 'L2']
    ,['Rollux', 'L2']
    ,['Manta', 'L2']
    ,['Kroma','L2']
    ,['Arbitrum Nova','L2']
    #L1
    ,['Ethereum', 'L1']
    #Others
    ,['Fantom', 'L1']
    ,['Avalanche', 'L1']
    ,['Gnosis' , 'L1']
    ,['Celo', 'L1']
    ,['Polygon', 'L1']
    ,['BSC', 'L1']
]

curated_protocols = [
         ['aevo', 'L2',['optimism','arbitrum','ethereum']]
        ,['dydx', 'L2',['ethereum']]
        ,['immutablex', 'L2',['ethereum']]
        ,['apex-protocol', 'L2',['ethereum']]
        # ,['brine.fi', 'L2',['ethereum']] #no longer listed
        ,['loopring', 'L2',['ethereum']]
        ,['aztec', 'L2',['ethereum']]
        ,['lyra-v2', 'L2', ['ethereum']]
    ]

In [7]:
opstack_metadata = pd.read_csv('../op_chains_tracking/outputs/chain_metadata.csv')
#filter to defillama chains
tvl_opstack_metadata = opstack_metadata[~opstack_metadata['defillama_slug'].isna()].copy()
# opstack_chains
tvl_opstack_metadata = tvl_opstack_metadata[['defillama_slug','chain_layer','chain_type']]

opstack_chains = tvl_opstack_metadata[~tvl_opstack_metadata['defillama_slug'].str.contains('protocols/')].copy()
op_superchain_chains = tvl_opstack_metadata[tvl_opstack_metadata['chain_type'].notna()].copy()
opstack_protocols = tvl_opstack_metadata[tvl_opstack_metadata['defillama_slug'].str.contains('protocols/')].copy()
#clean column
opstack_protocols['defillama_slug'] = opstack_protocols['defillama_slug'].str.replace('protocols/','')
# Use apply to create an array with one element
opstack_protocols['chain_list'] = opstack_protocols.apply(lambda x: ['ethereum'], axis=1) #guess, we can manually override if not

In [8]:
print(op_superchain_chains)

   defillama_slug chain_layer chain_type
0        Optimism          L2   standard
1            Base          L2   standard
2            Zora          L2   standard
17           Mode          L2   standard
29            BOB          L2   frontier
36        Fraxtal          L2   frontier
43   CyberConnect          L2   frontier


In [9]:
# Create new lists
chains = curated_chains.copy()
protocols = curated_protocols.copy()
# Iterate through the DataFrame and append data to 'chains'
for index, row in opstack_chains.iterrows():
    if all(row['defillama_slug'] != item[0] for item in chains):
        chains.append([row['defillama_slug'], row['chain_layer']])
# Iterate through the DataFrame and append data to 'chains'
for index, row in opstack_protocols.iterrows():
    if all(row['defillama_slug'] != item[0] for item in protocols):
        protocols.append([row['defillama_slug'], row['chain_layer'],row['chain_list']])


In [10]:
chains_df = pd.DataFrame(chains, columns = ['chain','layer'])
chains_df['is_EVM'] = True
chains_df['is_Rollup'] = True
# Get the chains already present in chains_df
existing_chains = set(chains_df['chain'])

# Filter out the chains from considered_chains that are not in chains_df
to_append = considered_chains[~considered_chains['chain'].isin(existing_chains)]

# Append the selected rows to chains_df
chains_df = pd.concat([chains_df, to_append], ignore_index=True)

# merged_chains_df.sort_values(by='chain').head(20)

In [11]:
chain_name_list = chains_df['chain'].unique().tolist()
get_app_list = op_superchain_chains['defillama_slug'].unique().tolist()

In [12]:
print('get tvls')
p = dfl.get_all_protocol_tvls_by_chain_and_token(min_tvl=min_tvl_to_count_apps, chains = get_app_list, do_aggregate = 'Yes')

get tvls


In [13]:
# p
# print('gen flows')
# p = dfl.generate_flows_column(p)
# p = dfl.get_tvl(apistring= 'https://api.llama.fi/protocol/velodrome', chains = chain_name_list, prot = 'velodrome',prot_name='Velodrome')

In [14]:
# Aggregate data
# TODO: Try to use config API to get Apps by Chain, since this only shows TVLs

app_dfl_list = p.groupby(['chain', 'date']).agg(
    distinct_protocol=pd.NamedAgg(column='protocol', aggfunc=pd.Series.nunique),
    distinct_parent_protocol=pd.NamedAgg(column='parent_protocol', aggfunc=pd.Series.nunique),
    sum_token_value_usd_flow=pd.NamedAgg(column='sum_token_value_usd_flow', aggfunc='sum'),
    sum_price_usd_flow=pd.NamedAgg(column='sum_price_usd_flow', aggfunc='sum'),
    sum_usd_value_check=pd.NamedAgg(column='sum_usd_value_check', aggfunc='sum')
)

p = None #Clear Memory

app_dfl_list = app_dfl_list.reset_index()
app_dfl_list = app_dfl_list.rename(columns={'chain':'defillama_slug'})


# display(app_dfl_list)
# app_dfl_list

In [15]:
p_agg = []
for p in protocols:
    try:
        d = dfl.get_single_tvl(p[0], p[2], print_api_str=False) #Set print_api_str=True for debugging
        # print(f"Raw content for {p}: {d}")
        d['chain_prot'] = p[0].title()
        d['layer'] = p[1]
        d['defillama_slug'] = 'protocols/' + p[0]
        d['source'] = 'protocol'
        # d['prot_chain'] = c
        p_agg.append(d)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for {p}: {e}")
        continue  # Continue to the next iteration of the loop
    except Exception as e:
        print(f"An unexpected error occurred for {p}: {e}")
        continue  # Continue to the next iteration of the loop

df = pd.concat(p_agg)
df = df.fillna(0)
# prob use total_app_tvl since it has more history and we're not yet doing flows
df_sum = df.groupby(['date', 'chain_prot', 'layer', 'defillama_slug', 'source']).agg({'usd_value':'sum'}).reset_index()
# print(df_sum.columns)

df_sum = df_sum[['date', 'usd_value', 'chain_prot', 'layer', 'defillama_slug', 'source']]
df_sum = df_sum.rename(columns={'chain_prot': 'chain', 'usd_value': 'tvl'})

# print(df_sum.tail(5))


In [16]:
# display(df_sum[df_sum['chain']=='Aevo'])
# display(df_sum)

In [17]:
c_agg = []
for c in chains:
        try:
                d = dfl.get_historical_chain_tvl(c[0])
                d['chain'] = c[0]
                d['layer'] = c[1]
                d['defillama_slug'] = c[0]
                d['source'] = 'chain'
                
                c_agg.append(d)
        except Exception as e:
                print(f"An unexpected error occurred for {c}: {e}")
                continue  # Continue to the next iteration of the loop


df_ch = pd.concat(c_agg)
df = pd.concat([df_ch,df_sum])
# Rename
df['chain'] = df['chain'].str.replace('%20', ' ', regex=False)
df['chain'] = df['chain'].str.replace('-', ' ', regex=False)
df.loc[df['chain'] == 'Optimism', 'chain'] = 'OP Mainnet'
df.loc[df['chain'] == 'BSC', 'chain'] = 'BNB'
df.loc[df['chain'] == 'Brine.Fi', 'chain'] = 'Brine'
df.loc[df['chain'] == 'Immutablex', 'chain'] = 'ImmutableX'


df = df[df['date'] <= current_utc_date ] #rm dupes at current datetime
df['tvl'] = df['tvl'].fillna(0)
df['tvl'] = df['tvl'].replace('', 0)

df['date'] = pd.to_datetime(df['date']) - timedelta(days=1) #map to the prior day, since dfl adds an extra day

error - historicalChainTvl api


In [18]:
df = df.merge(app_dfl_list, on =['defillama_slug','date'], how='left')

In [19]:
df.sample(5)

,date,tvl,chain,layer,defillama_slug,source,statusCode,body,headers,distinct_protocol,distinct_parent_protocol,sum_token_value_usd_flow,sum_price_usd_flow,sum_usd_value_check
19114,2021-02-01,0.000000e+00,Loopring,L2,protocols/loopring,protocol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6910,2024-01-13,2.156287e+07,opBNB,L2,opBNB,chain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,2022-12-11,5.561515e+08,OP Mainnet,L2,Optimism,chain,NaN,NaN,NaN,91.0,89.0,1.373115e+06,-1.579254e+07,6.228579e+08
5071,2022-03-28,4.454774e+07,Boba,L2,Boba,chain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280,2022-04-19,4.543562e+08,OP Mainnet,L2,Optimism,chain,NaN,NaN,NaN,30.0,30.0,2.941001e+06,8.438166e+06,4.464923e+08


In [20]:
# Add Metadata
meta_cols = ['defillama_slug', 'is_op_chain','mainnet_chain_id','op_based_version', 'alignment','chain_name','display_name']

df = df.merge(opstack_metadata[meta_cols], on='defillama_slug', how = 'left')

df['alignment'] = df['alignment'].fillna('Other EVMs')
df['is_op_chain'] = df['is_op_chain'].fillna(False)

In [21]:
cutoff_date = pd.Timestamp.now() - pd.Timedelta(days=365)
# Filter the DataFrame to the last 365 days
df_365 = df[df['date'] >= cutoff_date]

df_365.sample(5)

,date,tvl,chain,layer,defillama_slug,source,statusCode,body,headers,distinct_protocol,distinct_parent_protocol,sum_token_value_usd_flow,sum_price_usd_flow,sum_usd_value_check,is_op_chain,mainnet_chain_id,op_based_version,alignment,chain_name,display_name
11655,2023-09-23,4.970519e+07,Fantom,L1,Fantom,chain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,Other EVMs,NaN,NaN
14253,2024-04-14,2.941380e+08,Gnosis,L1,Gnosis,chain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,Other EVMs,NaN,NaN
18644,2024-03-10,8.470320e+08,Blast,L2,Blast,chain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,81457.0,modern,OP Stack Fork,blast,Blast
23119,2023-10-13,7.344704e+06,Aevo,L2,protocols/aevo,protocol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,legacy bedrock,OP Stack Fork,aevo,Aevo
6412,2023-06-09,4.145303e+07,Metis,L2,Metis,chain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1088.0,legacy ovm,OP Stack Fork,metis andromeda,Metis


In [22]:
#  Define aggregation functions for each column
aggregations = {
    'tvl': ['min', 'last', 'mean'],
    'distinct_protocol': ['min', 'last', 'mean'],
    'distinct_parent_protocol': ['min', 'last', 'mean'],
    'sum_token_value_usd_flow': 'sum',
    'sum_price_usd_flow': 'sum',
    'sum_usd_value_check': 'sum'
}
# Function to perform aggregation based on frequency
def aggregate_data(df, freq, date_col='date', groupby_cols=None, aggs=None):
    if groupby_cols is None:
        groupby_cols = ['chain', 'layer', 'defillama_slug', 'source', 'is_op_chain', 'mainnet_chain_id', 'op_based_version', 'alignment', 'chain_name', 'display_name']
    if aggs is None:
        aggs = aggregations

    # Group by the specified frequency and other columns, then apply aggregations
    df_agg = df.groupby([pd.Grouper(key=date_col, freq=freq, closed='left')] + groupby_cols, dropna=False).agg(aggs).reset_index()
    # Flatten the hierarchical column index and concatenate aggregation function names with column names
    df_agg.columns = [f'{col}_{func}' if func != '' else col for col, func in df_agg.columns]
    # Rename the 'date' column based on the frequency
    date_col_name = 'month' if freq == 'MS' else 'week'
    df_agg.rename(columns={f'{date_col}_': date_col_name}, inplace=True)

    return df_agg

# Perform monthly aggregation
df_monthly = aggregate_data(df, freq='MS')

# Perform weekly aggregation
df_weekly = aggregate_data(df, freq='W-MON')

# Now df_monthly and df_weekly contain the aggregated data

In [23]:
# df[df['chain'] == 'Ethereum'].tail(5)
# df_monthly.sample(10)
# df_weekly.sample(10)

,date,chain,layer,defillama_slug,source,is_op_chain,mainnet_chain_id,op_based_version,alignment,chain_name,...,tvl_mean,distinct_protocol_min,distinct_protocol_last,distinct_protocol_mean,distinct_parent_protocol_min,distinct_parent_protocol_last,distinct_parent_protocol_mean,sum_token_value_usd_flow_sum,sum_price_usd_flow_sum,sum_usd_value_check_sum
655,2021-11-08,Dydx,L2,protocols/dydx,protocol,False,NaN,NaN,Other EVMs,NaN,...,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2892,2024-01-15,ImmutableX,L2,protocols/immutablex,protocol,False,NaN,NaN,Other EVMs,NaN,...,3.751254e+07,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1948,2023-05-08,Loopring,L2,protocols/loopring,protocol,False,NaN,NaN,Other EVMs,NaN,...,1.120625e+08,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2637,2023-11-20,Arbitrum,L2,Arbitrum,chain,False,NaN,NaN,Other EVMs,NaN,...,2.042839e+09,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
407,2021-06-14,BNB,L1,BSC,chain,False,NaN,NaN,Other EVMs,NaN,...,1.100462e+10,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1994,2023-05-22,Polygon zkEVM,L2,Polygon zkEVM,chain,False,NaN,NaN,Other EVMs,NaN,...,7.978574e+06,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1076,2022-05-23,ImmutableX,L2,protocols/immutablex,protocol,False,NaN,NaN,Other EVMs,NaN,...,5.823897e+07,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1779,2023-03-13,Arbitrum,L2,Arbitrum,chain,False,NaN,NaN,Other EVMs,NaN,...,1.713372e+09,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
223,2021-01-04,Polygon,L1,Polygon,chain,False,NaN,NaN,Other EVMs,NaN,...,3.238893e+05,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
747,2021-12-20,Metis,L2,Metis,chain,False,1088.0,legacy ovm,OP Stack Fork,metis andromeda,...,2.787130e+05,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [24]:
# export
folder = 'outputs/'
df.to_csv(folder + 'dfl_chain_tvl.csv', index = False)
df_365.to_csv(folder + 'dfl_chain_tvl_t365d.csv', index = False)
df_monthly.to_csv(folder + 'dfl_chain_tvl_monthly.csv', index = False)
df_weekly.to_csv(folder + 'dfl_chain_tvl_weekly.csv', index = False)
# Write to Dune
du.write_dune_api_from_pandas(df, 'dfl_chain_tvl',\
                             'TVL for select chains from DefiLlama')
du.write_dune_api_from_pandas(df_monthly, 'dfl_chain_tv_monthly',\
                             'Monthly TVL for select chains from DefiLlama')
du.write_dune_api_from_pandas(df_weekly, 'dfl_chain_tv_weekly',\
                             'Weekly TVL for select chains from DefiLlama')

table at: dune.oplabspbc.dataset_dfl_chain_tvl
Response status code: 200
Response content: b'{"success":true,"table_name":"dfl_chain_tvl"}'
table at: dune.oplabspbc.dataset_dfl_chain_tv_monthly
Response status code: 200
Response content: b'{"success":true,"table_name":"dfl_chain_tv_monthly"}'
table at: dune.oplabspbc.dataset_dfl_chain_tv_weekly
Response status code: 200
Response content: b'{"success":true,"table_name":"dfl_chain_tv_weekly"}'
